# KOL主题分析和词频统计
四个主题
- Quality Movement
- Function/Tech
- Swiss/Professional
- sports


In [1]:
import pandas as pd

import numpy as np
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from matplotlib import font_manager

from IPython.display import display, HTML
import matplotlib as mpl 
from matplotlib.font_manager import *   
import openpyxl

import seaborn as sns
sns.set_style("whitegrid")

from itertools import cycle, islice

title_fsize = 15
brand_colors  = ['blue', 'darkorange', 'grey', 'yellow']
store_colors = ['red', 'purple', 'green']
%matplotlib inline
mpl.style.use('default')
#指定默认字体  
myfont = FontProperties(fname="C:\Windows\Fonts\msyh.ttc")  


In [2]:
fn = "../data/201804/Tissot KiSS_Data Capturing_201804.xlsx"
# wb = openpyxl.load_workbook(fn)
# sht_names = wb.sheetnames
# wb.close()

In [3]:
## 读文件
brands = ['tissot','fiyta','casio','longines']
brand_index = ['TISSOT', 'FIYTA', 'CASIO', 'LONGINES']
cols = ['id','type','pub_dt','pub_time','author','media_type','source','title','body']
df_kol = None


for br_ in brand_index:
    
    if len(br_)>0:
        df_sht = pd.read_excel(fn,br_, parse_cols="A:I", names=cols)
        df_sht['brand'] = br_
        print(df_sht.shape)
        df_kol = pd.concat([df_kol,df_sht], axis =0)


## fix data error
df_kol.brand = df_kol.brand.str.upper()
print('sample data:')
df_kol.head() 


(12032, 10)
(2308, 10)
(6180, 10)
(25169, 10)
sample data:


,id,type,pub_dt,pub_time,author,media_type,source,title,body,brand
0,1,租售类,2018-03-01,08:53:28,NaN,新闻,百度网页,白山市周边欧米茄手表回收商家二手手表回收价格_志趣网,发布时间 2018-03-01 08:35:36 国家地区 中国» 武汉 » 江汉 发...,TISSOT
1,2,租售类,2018-03-01,10:14:03,NaN,新闻,百度网页,济南市周边欧米茄手表回收商家二手手表回收价格_志趣网,发布时间 2018-03-01 09:52:46 国家地区 中国» 武汉 » 江汉 发...,TISSOT
2,3,租售类,2018-03-01,09:43:43,NaN,新闻,百度网页,景德镇市周边欧米茄手表回收商家二手手表回收价格_志趣网,发布时间 2018-03-01 09:42:45 国家地区 中国» 武汉 » 江汉 发...,TISSOT
3,4,租售类,2018-03-01,09:17:28,NaN,新闻,百度网页,合肥市周边欧米茄手表回收商家二手手表回收价格_志趣网,发布时间 2018-03-01 09:17:13 国家地区 中国» 武汉 » 江汉 发...,TISSOT
4,5,租售类,2018-03-01,09:05:28,NaN,新闻,百度网页,宿迁市周边欧米茄手表回收商家二手手表回收价格_志趣网,发布时间 2018-03-01 09:05:15 国家地区 中国» 武汉 » 江汉 发...,TISSOT


In [4]:
df_kol.groupby(['brand','type'])['id'].agg('size')

brand     type
CASIO     仿品类       843
          常规       4173
          租售类       830
          维修类       334
FIYTA     仿品类       145
          常规       1662
          租售类       330
          维修类       171
LONGINES  仿品类      3104
          常规       3523
          租售类     15646
          维修类      2896
TISSOT    仿品类      1876
          常规       3900
          租售类      4822
          维修类      1434
Name: id, dtype: int64

In [5]:
df_title= df_kol[ (df_kol['type']=='常规') &(df_kol['media_type']=='新闻')  ]
df_title.title = df_title.title.str.replace('&#\d+;','')
df_title.groupby(['brand','type'])['id'].agg('size')


c:\conda64\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


brand     type
CASIO     常规      3395
FIYTA     常规      1277
LONGINES  常规      2069
TISSOT    常规      2018
Name: id, dtype: int64

In [6]:
# 对正文分词
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse as jieba_analyse
import jieba.posseg as pseg

jieba_analyse.set_stop_words('stopwords.txt') 
jieba.load_userdict('userdict.txt')

# jieba.add_word('中大型')
datas = []

df_wcut = df_title.assign(text = lambda o: o.title)['text']
corpus = df_wcut.values.astype(str)

for oneline in corpus:
    seg_list = pseg.cut(oneline.strip()[:20])
#     print([",".join(w) for w in seg_list ])
    words = [w for w,c in seg_list if len(w)>1 and not w.isnumeric()]
    
    datas.append(" ".join(words))
np.array(datas).shape

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yu.pan\AppData\Local\Temp\jieba.cache
Loading model cost 0.813 seconds.
Prefix dict has been built succesfully.


(8759,)

In [7]:
# 计算tfidf
freWord = CountVectorizer()
wordFreqArr = freWord.fit_transform(datas)


In [8]:
from sklearn.cluster import dbscan
core_samples,labels   = dbscan(wordFreqArr, eps=0.01, min_samples=1)


In [9]:
df_title.loc[:,'doc_id'] =labels
df_title.doc_id.value_counts()[:5]


c:\conda64\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
c:\conda64\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


401     72
405     64
1669    47
1508    34
155     28
Name: doc_id, dtype: int64

In [11]:
df_doc = df_title.groupby(['doc_id']).head(1)[['doc_id', 'title','brand']]
df_doc_cnt  = df_title.groupby(['doc_id'])['doc_id'].count().to_frame()

df_doc_cnt.columns = ['issues']
df_doc = df_doc.join(df_doc_cnt,on='doc_id', rsuffix='_r', how='inner')

for b_ in df_doc.brand.unique():
    print("Top 20 news: "+ b_)
    df_t20 = df_doc[df_doc.brand==b_].sort_values('issues', ascending = False).head(20)
    display(df_t20)

Top 20 news: TISSOT


,doc_id,title,brand,issues
3460,401,成都：女子将20只手表捆绑藏匿腰间被海关挡获,TISSOT,72
3480,405,女子将20只手表绑腰间入关 箱内还有大量韩国衣帽,TISSOT,64
1336,155,"学生哥凑了好久买块手表,2000选天梭还是卡西欧? - 第14页 - 步行...",TISSOT,28
10020,1204,想玩转人群精细化运营?品牌不可错过天猫超级粉丝日!_搜狐科技_...,TISSOT,28
9910,1189,"天猫超级粉丝日又来了,这次玩嗨的是天梭的粉丝们",TISSOT,26
9908,1187,"天猫超级粉丝日：精准触达,蓄力粉丝的能量（1）",TISSOT,23
9332,1103,短短几天的巴塞尔表展 让王大陆乔欣都变成了铁杆表迷_搜狐时尚_...,TISSOT,14
2194,285,这一刻享受时光之旅 寻天梭表的历史印记感悟瑞士的绮丽..._网易新闻,TISSOT,14
2134,276,天津旺典表业总经理张凯效:坦诚经营 做跟时间赛跑的商人,TISSOT,14
709,89,百大购物中心马鞍山路店-促销快讯,TISSOT,13


Top 20 news: FIYTA


,doc_id,title,brand,issues
747,1669,消费升级名表行业持续复苏 飞亚达去年营收利润创新高,FIYTA,47
342,1508,"巴塞尔表展在即,这些抢先登场的腕表将是市场风向标（上）",FIYTA,34
1334,1874,飞亚达手表怎么样？,FIYTA,19
582,1603,飞亚达A年报净利增长27% 拟10派2元,FIYTA,17
28,1453,这个钟表品牌把明星IP玩得溜溜转,FIYTA,17
121,1465,京东时尚丁霞:京东要做有质感的时尚电商,FIYTA,16
16,1448,"京东时尚“质感”升级,成为时尚产业新T台_搜狐时尚_搜狐网",FIYTA,15
14,1447,2018第19届深圳国际机械制造工业展览会,FIYTA,12
752,1671,消费升级名表行业复苏 飞亚达去年营收利润创新高,FIYTA,10
1615,1972,飞亚达手表质量怎么样,FIYTA,9


Top 20 news: CASIO


,doc_id,title,brand,issues
5523,4057,卡西欧亮相2018巴塞尔国际钟表珠宝展,CASIO,27
3844,3591,发力时尚生活消费 苏宁“3+2”战略是什么?_装修资..._搜狐焦点家居,CASIO,25
2836,3219,卡西欧PRO TREK户外运动手表新品PRW-6600Y外型挺好...,CASIO,19
324,2293,“新零售”创新发展模式成为中国经济增长重要支撑_生活新财经头条...,CASIO,16
2235,2982,"驴妈妈春季旅游节启动 线路最高立减1200元,出境游限时半价抢-商业...",CASIO,16
5179,3966,卡西欧推出限量版智能手表 仅售700只,CASIO,14
3282,3389,卡西欧PRO TREK户外运动手表新款PRW-6600Y系列有几...,CASIO,14
2783,3194,第十五届中国零售业发展高峰论坛在上海顺利召开_资讯_中国时尚...,CASIO,10
43,2181,香港化妆品:植村秀shu uemura最新限定版底妆组合(附价格),CASIO,9
2900,3236,户外运动手表新品，卡西欧的PRW-6600Y系列和颂拓的...,CASIO,9


Top 20 news: LONGINES


,doc_id,title,brand,issues
11636,4645,郭富城亮相浪琴周年巡展 同台美女主持惹关注,LONGINES,13
12731,4728,外资钟表、珠宝首饰品牌排行榜：劳力士最佳,LONGINES,10
19110,5068,“顶”级上海马术冠军赛:让IP赛事成为城市“公交汽车”_搜..._搜狐,LONGINES,9
4788,4390,浪琴戴卓维纳腕表 以卓越的品质和精湛的技术受到世代人的认可-第2页,LONGINES,8
13435,4768,《红海行动》不仅有士兵与硝烟 还有军工表的前世今生_凤凰时尚,LONGINES,7
17667,4990,浪琴_浪琴表2018年巴塞尔表展新品: 开创者系列闪耀金色光泽|腕表...,LONGINES,7
23107,5244,"国外品牌名翻译而论,哪些译名可以称作神翻译? - 第18页 - 步行街...",LONGINES,6
16437,4940,五星级马术赛事落户上海 高大上观赛感受等您体验_网易体育,LONGINES,6
22018,5193,【头条】2018上海浪琴环球马术冠军赛首推“骑士看台”:..._搜狐体育,LONGINES,6
16613,4946,上海环球马术冠军赛4月开战 团体冠军赛成亮点,LONGINES,5


In [15]:
display(HTML('<b>sample news tile in 1 category:</b>'))
df_title.loc[df_title.doc_id == 401,'title']


3460                      成都：女子将20只手表捆绑藏匿腰间被海关挡获
3467             成都:女子将20只手表捆绑藏匿腰间被海关挡获-阳新新闻-阳新网
3512           成都:女子将20只手表捆绑藏匿腰间被海关挡获_新闻频道_中国青年网
3514              成都:女子将20只手表捆绑藏匿腰间被海关挡获 - 咸宁新闻网
3533                      成都:女子将20只手表捆绑藏匿腰间被海关挡获
3541             成都:女子将20只手表捆绑藏匿腰间被海关挡获_社会新闻_大众网
3542                  成都:女子将20只手表捆绑藏匿腰间被海关挡获-中新网
3546               成都:女子将20只手表捆绑藏匿腰间被海关挡获_社会_环球网
3547             成都:女子将20只手表捆绑藏匿腰间被海关挡获_搜狐新闻_搜狐网
3560             成都:女子将20只手表捆绑藏匿腰间被海关挡获_搜狐社会_搜狐网
3564                  成都:女子将20只手表捆绑藏匿腰间被海关挡获-泉州网
3565                      成都:女子将20只手表捆绑藏匿腰间被海关挡获
3648                      成都：女子将20只手表捆绑藏匿腰间被海关挡获
3649                      成都：女子将20只手表捆绑藏匿腰间被海关挡获
3650                      成都：女子将20只手表捆绑藏匿腰间被海关挡获
3680                成都:女子将20只手表捆绑藏匿腰间被海关挡获_兵马俑在线
3714                成都:女子将20只手表捆绑藏匿腰间被海关挡获 - 听热点
3715      成都:女子将20只手表捆绑藏匿腰间被海关挡获 - CHN军网 - CHN军网
3735                  成都:女子将20只手表捆绑藏匿腰间被海关挡获-沃门户
3813        成都:女子将20只手表捆绑藏匿腰间被海关挡获|手表|海关..._新浪新闻
3815                